**Mounting the google drive project folder**

In [2]:
from google.colab import drive
import os
drive.mount('/content/drive/', force_remount=True)
os.chdir('/content/drive/My Drive/Lifelogging/')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive/


**Funtion to determine the blur factor of an image**

In [5]:
def blur(image):
  def variance_of_laplacian(image):
    #calculating and returning the Laplacian variance of the image
    return cv2.Laplacian(image, cv2.CV_64F).var()

  #converting the image to grayscale for better processing
  gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
  v = variance_of_laplacian(gray)
  text = "Clear"

  if v < 100:
      text = "Blurry"

  #adding blur detail onto the image for intermediate result
  #only if chose to display
  cv2.putText(image, "{}: {:.2f}".format(text, v), (10, 30), cv2.FONT_HERSHEY_PLAIN, 1.7, (250, 0, 0), 2)

  #from google.colab.patches import cv2_imshow
  #cv2_imshow(image)
  return v

**Funtion to determine the noise factor of an image**

In [6]:
def noise(image):

  #calculating average signal of the image foreground 
  m = np.mean(image)          

  #calculating the stand deviation image background
  s = np.std(image)           

  #Signal-to-Noise ratio
  SNR = 10*math.log10(m/s)    

  return SNR

**Function to determine the contrast value of an image**

In [7]:
def contrast(image):
  
  #converting the image to grayscale
  gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

  #calculating the variance of the grayscale image
  d = gray.var()

  return d

**The principal code block that initiates image processing**

In [10]:
#importing necessary libraries for the project
import pathlib
import cv2
import numpy as np
import math
import pandas as pd
import glob

#data_dir = pathlib.Path('/content/drive/My Drive/')

#traversing through the parent folder to determine the total amount of images existing across subfolders
image_count = len(list(glob.glob('/content/drive/My Drive/Lifelogging/**/*.JPG')))
print('Images found\t:',image_count)
print('----------------------------------------------------\n')


#defining the columns of dataframe used for image dataset
image_set = pd.DataFrame(columns = ['Filename','Blur','Noise','Contrast'])

#extracting the list of all image files under the project folder
files = glob.glob('/content/drive/My Drive/Lifelogging/**/*.JPG')

#looping through all the files
for filename in files:
  #converting the path type to string type
  #reading an image file
  image = cv2.imread(str(filename))
  
  #extracting the filename
  base=os.path.basename(str(filename))
  
  #obtaining the blur, noise and contrast values from the distinct functions for the respective image
  blur_count = blur(image)
  noise_count = noise(image)
  contrast_count = contrast(image)
  
  #adding the attribute values to the dataframe
  image_set.loc[len(image_set)] = [base,blur_count,noise_count,contrast_count]

#checking the resultant dataframe
image_set.head()
image_set.tail()

Images found	: 34940
----------------------------------------------------



,Filename,Blur,Noise,Contrast
0,B00004290_21I6X0_20180504_130739E.JPG,89.212916,-0.325969,8744.913871
1,B00004951_21I6X0_20180504_173845E.JPG,181.041884,1.938497,2807.010122
2,B00004150_21I6X0_20180504_120752E.JPG,70.695700,1.470455,7859.658589
3,B00003796_21I6X0_20180504_093517E.JPG,82.724502,2.321314,3314.085188
4,B00003685_21I6X0_20180504_084651E.JPG,234.090913,0.212817,7392.070857


**Determining the maximum and minimum values of each attributes existing in the dataset** 

In [ ]:
max_blur = image_set[['Blur']].idxmax()
max_noise = image_set[['Noise']].idxmax()
max_contrast = image_set[['Contrast']].idxmax()

min_blur = image_set[['Blur']].idxmin()
min_noise = image_set[['Noise']].idxmin()
min_contrast = image_set[['Contrast']].idxmin()

print('Blur\n--------------\n',image_set.iloc[max_blur],'\n',image_set.iloc[min_blur],'\n')
print('SNR\n--------------\n',image_set.iloc[max_noise],'\n',image_set.iloc[min_noise],'\n')
print('Contrast\n--------------\n',image_set.iloc[max_contrast],'\n',image_set.iloc[min_contrast],'\n')

**Exporting the dataset as .csv**

In [12]:
from google.colab import files
image_set.to_csv('Image_data.csv') 
files.download('Image_data.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>